In [1]:
import cv2 as cv
import os
video = cv.VideoCapture("my_video.mp4")

In [2]:

# Display relative video propreties
width = int(video.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv.CAP_PROP_FRAME_HEIGHT))
frame_rate = video.get(cv.CAP_PROP_FPS)
frame_count = int(video.get(cv.CAP_PROP_FRAME_COUNT))
duration = frame_count / frame_rate

print("Video frame width: \t  ", width)
print("Video frame height: \t  ", height)
print("Video frame rate property:", frame_rate)
print("Video frame count: \t  ", frame_count)
print("Video duration (s): \t  ", duration)

Video frame width: 	   854
Video frame height: 	   480
Video frame rate property: 23.976023976023978
Video frame count: 	   136
Video duration (s): 	   5.672333333333333


2. Utilisez la fonction read pour extraire les frames de votre vidéo. Ensuite, sauvegardez les dans le dossier de travail à l’aide de la fonction imwrite

In [3]:
# Création du dossier pour stocker les frames
if not os.path.exists('frames'):
    os.makedirs('frames')

frame_nb = 0
frames_left = True

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break

    frame_name = os.path.join('frames', f"frame_{frame_nb:04d}.jpg")
    cv.imwrite(frame_name, frame)  
    frame_nb += 1

print(f"{frame_nb} frames sauvegardées.")
video.release()

136 frames sauvegardées.


question 3

In [4]:
video = cv.VideoCapture('my_video.mp4')
frame_nb = 0

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    
    cv.imshow('Frame', frame)  
    frame_nb += 1
    
    # Attente de 50ms entre chaque frame
    key = cv.waitKey(50)
    
    # S'arrêter avec la touche 'q'
    if key == ord('q'):
        break

video.release()
cv.destroyAllWindows()

question 4

In [5]:
video.release()  
cv.destroyAllWindows()  

convention en niveaux de gris

In [7]:
if not os.path.exists('grayscale_frames'):
    os.makedirs('grayscale_frames')

video = cv.VideoCapture('my_video.mp4')
frame_nb = 0

fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('grayscale_video.avi', fourcc, frame_rate, 
                     (width, height), isColor=False)

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    
    # Conversion BGR vers niveaux de gris
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # Sauvegarde du frame en niveaux de gris
    frame_name = os.path.join('grayscale_frames', f"frame_{frame_nb:04d}.jpg")
    cv.imwrite(frame_name, gray)
    
    out.write(gray)
    
    cv.imshow('Frame Original', frame)
    cv.imshow('Frame Niveaux de Gris', gray)
    frame_nb += 1
    
    key = cv.waitKey(50)

video.release()
out.release()
cv.destroyAllWindows()

Algorithme de seuillage globale

In [ ]:
frame = cv.imread('frames/frame_0089.jpg')
gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

# Test avec différents seuils d'initialisation
Thresholds = [15, 120, 240]

for T_init in Thresholds:
    print(f"Seuil initial T₀ = {T_init}")
    T = T_init
    
    for iteration in range(5):
        # Seuillage binaire
        _, binary_frame = cv.threshold(gray_frame, T, 255, cv.THRESH_BINARY)
        
        # Calcul des moyennes des deux classes
        mean1 = gray_frame[binary_frame == 255].mean()  # Pixels blancs (> T)
        mean2 = gray_frame[binary_frame == 0].mean()     # Pixels noirs (≤ T)
        
        # Nouveau seuil
        T_new = (mean1 + mean2) / 2
        
        print(f"  Itération {iteration+1}: T = {T:.4f}")
        
        # Affichage optionnel
        # cv.imshow('Binary Frame', binary_frame)
        # cv.waitKey(500)
        
        T = T_new
    
    print(f"  Seuil final : T_final = {T:.4f}\n")

cv.destroyAllWindows()

=== Algorithme de Seuillage Global Heuristique ===

Seuil initial T₀ = 15
  Itération 1: T = 15.0000
  Itération 2: T = 66.7674
  Itération 3: T = 93.6191
  Itération 4: T = 94.3681
  Itération 5: T = 94.3909
  Seuil final : T_final = 94.3909

Seuil initial T₀ = 120
  Itération 1: T = 120.0000
  Itération 2: T = 95.0096
  Itération 3: T = 94.4096
  Itération 4: T = 94.3909
  Itération 5: T = 94.3909
  Seuil final : T_final = 94.3909

Seuil initial T₀ = 240
  Itération 1: T = 240.0000
  Itération 2: T = 156.0672
  Itération 3: T = 98.0923
  Itération 4: T = 94.4791
  Itération 5: T = 94.3909
  Seuil final : T_final = 94.3909



Methode d'otsu 

code d'implementation

In [10]:
# Test sur un seul frame
frame = cv.imread('frames/frame_0089.jpg')
gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

# Seuillage d'Otsu
_, binary_frame = cv.threshold(gray_frame, 0, 255, 
                               cv.THRESH_BINARY + cv.THRESH_OTSU)

cv.imshow('Frame Original (Niveaux de Gris)', gray_frame)
cv.imshow('Segmentation Otsu', binary_frame)
cv.waitKey(0)
cv.destroyAllWindows()

Application sur toute la video 

In [ ]:
video = cv.VideoCapture('my_video.mp4')

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    
    # Conversion en niveaux de gris
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # Segmentation d'Otsu
    _, binary_frame = cv.threshold(gray_frame, 0, 255, 
                                   cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    cv.imshow('Vidéo Originale', frame)
    cv.imshow('Vidéo Segmentée (Otsu)', binary_frame)
    
    key = cv.waitKey(50) & 0xFF
    if key == ord('q'):
        break

video.release()
cv.destroyAllWindows()

: 

Segmentation temporelle 

In [ ]:
video = cv.VideoCapture('grayscale_video.avi')
lambda_val = 94.4258  # Seuil pour binariser la différence
frames_left, old_frame = video.read()

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    # Calcul de la différence absolue
    difference = cv.absdiff(old_frame, frame)
    # Seuillage de la différence
    _, difference_binary = cv.threshold(difference, lambda_val, 255, 
                                        cv.THRESH_BINARY)
    cv.imshow('Frame Courant', frame)
    cv.imshow('Différence (Mouvement Détecté)', difference_binary)
    

    old_frame = frame
    
    key = cv.waitKey(50) & 0xFF
    if key == ord('q'):
        break

video.release()
cv.destroyAllWindows()

: 

Differentiation de 3 frames 

In [12]:

folder = 'grayscale_frames'
grayscale_frames = sorted(os.listdir(folder))
lambda_val = 94.4258

# Créer un dossier pour sauvegarder les résultats
os.makedirs('comparaison_N', exist_ok=True)

# Frame de référence pour comparaison
frame_ref = 90

N_values = [5, 12, 23, 30, 40 , 45]

for N in N_values:
    i = frame_ref
    
    # Vérification des bornes
    if i - N < 0 or i + N >= len(grayscale_frames):
        print(f"N={N} : Impossible (hors limites)")
        continue
    
    # Chargement des trois frames
    img = cv.imread(os.path.join(folder, grayscale_frames[i]), cv.IMREAD_GRAYSCALE)
    img_after = cv.imread(os.path.join(folder, grayscale_frames[i + N]), cv.IMREAD_GRAYSCALE)
    img_before = cv.imread(os.path.join(folder, grayscale_frames[i - N]), cv.IMREAD_GRAYSCALE)
    
    # Calcul des différences
    diff1 = cv.absdiff(img_after, img)
    _, diff1 = cv.threshold(diff1, lambda_val, 255, cv.THRESH_BINARY)
    
    diff2 = cv.absdiff(img, img_before)
    _, diff2 = cv.threshold(diff2, lambda_val, 255, cv.THRESH_BINARY)
    
    # Intersection
    result = cv.bitwise_and(diff1, diff2)
    
    # SAUVEGARDE des résultats
    cv.imwrite(f'comparaison_N/N_{N}_detection.jpg', result)
    cv.imwrite(f'comparaison_N/N_{N}_original.jpg', img)
    
    print(f"Sauvegardé : N={N} (frame {frame_ref})")

print("Terminé ! Vérifiez le dossier 'comparaison_N/'")

Sauvegardé : N=5 (frame 90)
Sauvegardé : N=12 (frame 90)
Sauvegardé : N=23 (frame 90)
Sauvegardé : N=30 (frame 90)
Sauvegardé : N=40 (frame 90)
Sauvegardé : N=45 (frame 90)
Terminé ! Vérifiez le dossier 'comparaison_N/'


Adaptive background subtraction

In [13]:
video = cv.VideoCapture('my_video.mp4')
lambda_val = 94.4258
frame_rate = video.get(cv.CAP_PROP_FPS)

# TESTER différentes valeurs de α
alpha_values = [0.01, 0.05, 0.1, 0.5, 0.9, 0.96, 0.99]

for alpha in alpha_values:
    print(f"\n=== Test avec α = {alpha} ===")
    
    # Réinitialiser la vidéo
    video.set(cv.CAP_PROP_POS_FRAMES, 0)
    
    frames_left, background = video.read()
    background = cv.cvtColor(background, cv.COLOR_BGR2GRAY).astype(float)
    
    frame_count = 0
    
    while True:
        frames_left, frame = video.read()
        if not frames_left:
            break
        
        gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY).astype(float)
        
        # Différence avec arrière-plan
        diff = cv.absdiff(gray_frame, background)
        _, diff_binary = cv.threshold(diff.astype('uint8'), lambda_val, 255, 
                                      cv.THRESH_BINARY)
        
        # Mise à jour arrière-plan
        background = alpha * gray_frame + (1 - alpha) * background
        
        # Affichage
        cv.imshow(f'α={alpha} - Détection', diff_binary)
        cv.imshow(f'α={alpha} - Background', background.astype('uint8'))
        cv.imshow('Frame Original', frame)
        
        frame_count += 1
        
        key = cv.waitKey(50) & 0xFF
        if key == ord('q'):
            break
    
    cv.destroyAllWindows()
    print(f"Traité {frame_count} frames avec α={alpha}")

video.release()


=== Test avec α = 0.01 ===
Traité 135 frames avec α=0.01

=== Test avec α = 0.05 ===
Traité 135 frames avec α=0.05

=== Test avec α = 0.1 ===
Traité 135 frames avec α=0.1

=== Test avec α = 0.5 ===
Traité 135 frames avec α=0.5

=== Test avec α = 0.9 ===
Traité 135 frames avec α=0.9

=== Test avec α = 0.96 ===
Traité 135 frames avec α=0.96

=== Test avec α = 0.99 ===
Traité 135 frames avec α=0.99


In [14]:
import cv2 as cv
import numpy as np
import os

os.makedirs('alpha_comparison', exist_ok=True)

video_path = 'my_video.mp4'
lambda_val = 94.4258
alpha_values = [0.01, 0.1, 0.96, 0.99]
frame_to_capture = 100  # Frame de référence

for alpha in alpha_values:
    video = cv.VideoCapture(video_path)
    
    _, background = video.read()
    background = cv.cvtColor(background, cv.COLOR_BGR2GRAY).astype(float)
    
    for i in range(frame_to_capture + 1):
        ret, frame = video.read()
        if not ret:
            break
        
        gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY).astype(float)
        
        diff = cv.absdiff(gray_frame, background)
        _, diff_binary = cv.threshold(diff.astype('uint8'), lambda_val, 255, 
                                      cv.THRESH_BINARY)
        
        background = alpha * gray_frame + (1 - alpha) * background
        
        # Sauvegarder à la frame cible
        if i == frame_to_capture:
            combined = np.hstack([
                gray_frame.astype('uint8'),
                diff_binary,
                background.astype('uint8')
            ])
            
            cv.putText(combined, f'alpha={alpha}', (10, 30), 
                      cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            cv.imwrite(f'alpha_comparison/alpha_{alpha:.2f}.jpg', combined)
            print(f"Sauvegardé: alpha_{alpha:.2f}.jpg")
    
    video.release()

print("\nOuvrez les images dans 'alpha_comparison/' pour voir les différences !")

Sauvegardé: alpha_0.01.jpg
Sauvegardé: alpha_0.10.jpg
Sauvegardé: alpha_0.96.jpg
Sauvegardé: alpha_0.99.jpg

Ouvrez les images dans 'alpha_comparison/' pour voir les différences !


In [2]:
import cv2 as cv
import numpy as np

video_path = 'my_video.mp4'
lambda_val = 94.4258

# Valeurs de α à tester
alpha_values = [0.01, 0.05, 0.1, 0.2, 0.5, 0.96]

# Ouvrir une vidéo pour chaque α
videos = [cv.VideoCapture(video_path) for _ in alpha_values]
backgrounds = []

# Initialisation des backgrounds
for video in videos:
    _, bg = video.read()
    backgrounds.append(cv.cvtColor(bg, cv.COLOR_BGR2GRAY).astype(float))

print("Appuyez sur 'q' pour quitter")

while True:
    rows = []
    all_frames_read = True
    
    for i, (video, alpha) in enumerate(zip(videos, alpha_values)):
        ret, frame = video.read()
        if not ret:
            all_frames_read = False
            break
        
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY).astype(float)
        
        # Calcul de la différence
        diff = cv.absdiff(gray, backgrounds[i])
        _, binary = cv.threshold(diff.astype('uint8'), lambda_val, 255, cv.THRESH_BINARY)
        
        # Mise à jour du background
        backgrounds[i] = alpha * gray + (1 - alpha) * backgrounds[i]
        
        # Compter pixels de mouvement
        motion_pixels = np.sum(binary > 0)
        
        # Créer une ligne avec les 3 images côte à côte
        row = np.hstack([
            gray.astype('uint8'),
            binary,
            backgrounds[i].astype('uint8')
        ])
        
        # Ajouter les labels et statistiques
        cv.putText(row, f'alpha={alpha}', (10, 30), 
                  cv.FONT_HERSHEY_SIMPLEX, 0.7, (255), 2)
        cv.putText(row, f'{motion_pixels} px', (10, 60), 
                  cv.FONT_HERSHEY_SIMPLEX, 0.5, (255), 1)
        
        # Ajouter labels des colonnes (seulement pour la première ligne)
        if i == 0:
            cv.putText(row, 'FRAME', (gray.shape[1]//2 - 50, row.shape[0] - 10), 
                      cv.FONT_HERSHEY_SIMPLEX, 0.5, (200), 1)
            cv.putText(row, 'DETECTION', (gray.shape[1] + gray.shape[1]//2 - 70, row.shape[0] - 10), 
                      cv.FONT_HERSHEY_SIMPLEX, 0.5, (200), 1)
            cv.putText(row, 'BACKGROUND', (2*gray.shape[1] + gray.shape[1]//2 - 80, row.shape[0] - 10), 
                      cv.FONT_HERSHEY_SIMPLEX, 0.5, (200), 1)
        
        rows.append(row)
    
    if not all_frames_read or len(rows) < len(alpha_values):
        print("Fin de la vidéo")
        break
    
    # Empiler toutes les lignes verticalement
    grid = np.vstack(rows)
    
    # Ajouter un titre général
    title_bar = np.zeros((50, grid.shape[1]), dtype=np.uint8)
    cv.putText(title_bar, 'Comparaison des valeurs de alpha', 
              (grid.shape[1]//2 - 200, 35), cv.FONT_HERSHEY_SIMPLEX, 1, (255), 2)
    
    final_display = np.vstack([title_bar, grid])
    
    cv.imshow('Comparaison Alpha - Adaptive Background Subtraction', final_display)
    
    key = cv.waitKey(50) & 0xFF
    if key == ord('q'):
        break

# Libération
for video in videos:
    video.release()
cv.destroyAllWindows()
cv.waitKey(1)

print("Terminé!")

Appuyez sur 'q' pour quitter
Fin de la vidéo
Terminé!


In [3]:
alpha = 0.01
# Temps pour qu'un pixel "oublie" 90% de son influence
n = -np.log(0.1) / np.log(1 - alpha)
print(f"Mémoire : {n:.0f} frames = {n/24:.1f} secondes")

Mémoire : -229 frames = -9.5 secondes


In [4]:
import cv2 as cv
import numpy as np

video = cv.VideoCapture('my_video.mp4')
lambda_val = 94.4258

alphas = [0.005, 0.01, 0.02, 0.03, 0.05, 0.1]

for alpha in alphas:
    video.set(cv.CAP_PROP_POS_FRAMES, 0)
    
    _, bg = video.read()
    background = cv.cvtColor(bg, cv.COLOR_BGR2GRAY).astype(float)
    
    motion_pixels_list = []
    
    for i in range(200):  # 200 frames
        ret, frame = video.read()
        if not ret:
            break
        
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY).astype(float)
        diff = cv.absdiff(gray, background)
        _, binary = cv.threshold(diff.astype('uint8'), lambda_val, 255, cv.THRESH_BINARY)
        
        motion_pixels = np.sum(binary > 0)
        motion_pixels_list.append(motion_pixels)
        
        background = alpha * gray + (1 - alpha) * background
    
    avg = np.mean(motion_pixels_list)
    std = np.std(motion_pixels_list)
    
    print(f"α={alpha:.3f} | Avg: {avg:.0f} px | Std: {std:.0f} px | Score: {avg/std:.2f}")

video.release()

α=0.005 | Avg: 3998 px | Std: 3486 px | Score: 1.15
α=0.010 | Avg: 3836 px | Std: 3384 px | Score: 1.13
α=0.020 | Avg: 2900 px | Std: 2845 px | Score: 1.02
α=0.030 | Avg: 2458 px | Std: 2309 px | Score: 1.06
α=0.050 | Avg: 1915 px | Std: 1852 px | Score: 1.03
α=0.100 | Avg: 1170 px | Std: 1170 px | Score: 1.00


MOG2


In [ ]:
import cv2 as cv

video = cv.VideoCapture('my_video.mp4')

# Création du soustracteur MOG2
backSub_mog2 = cv.createBackgroundSubtractorMOG2(
    history=500,           # Historique de 500 frames
    varThreshold=16,       # Seuil de variance
    detectShadows=True     # Détection des ombres activée
)

frame_count = 0

while True:
    ret, frame = video.read()
    if not ret:
        break
    
    # Application du soustracteur MOG2
    fg_mask = backSub_mog2.apply(frame)
    
    # Affichage
    cv.imshow('Frame Original', frame)
    cv.imshow('MOG2 - Detection', fg_mask)
    
    frame_count += 1
    
    key = cv.waitKey(50) & 0xFF
    if key == ord('q'):
        break

video.release()
cv.destroyAllWindows()
cv.waitKey(1)

print(f"Traité {frame_count} frames avec MOG2")

Traité 136 frames avec MOG2


: 

KNN


In [ ]:
import cv2 as cv

video = cv.VideoCapture('my_video.mp4')

# Création du soustracteur KNN
backSub_knn = cv.createBackgroundSubtractorKNN(
    history=500,
    dist2Threshold=400.0,
    detectShadows=True
)

frame_count = 0

while True:
    ret, frame = video.read()
    if not ret:
        break
    
    # Application du soustracteur KNN
    fg_mask = backSub_knn.apply(frame)
    
    # Affichage
    cv.imshow('Frame Original', frame)
    cv.imshow('KNN - Detection', fg_mask)
    
    frame_count += 1
    
    key = cv.waitKey(50) & 0xFF
    if key == ord('q'):
        break

video.release()
cv.destroyAllWindows()
cv.waitKey(1)

print(f"Traité {frame_count} frames avec KNN")

Traité 136 frames avec KNN


: 